In [2]:
import pandas as pd

### Part 1: Reading the Data

In [3]:
df = pd.read_excel("heart_data1.xlsx")

In [4]:
print(df.head())

   Unnamed: 0  id    age  gender  height  weight  ap_hi  ap_lo  cholesterol  \
0         NaN   0  18393       2     168    62.0    110     80            1   
1         NaN   1  20228       1     156    85.0    140     90            3   
2         NaN   2  18857       1     165    64.0    130     70            3   
3         NaN   3  17623       2     169    82.0    150    100            1   
4         NaN   4  17474       1     156    56.0    100     60            1   

   gluc  smoke  alco  active  cardio  
0     1      0     0       1       0  
1     1      0     0       1       1  
2     1      0     0       0       1  
3     1      0     0       1       1  
4     1      0     0       0       0  


In [5]:
df = df.drop(df.columns[0], axis=1)

In [6]:
df = df.rename(columns={"cardio": "CVD"})

In [7]:
print(df.head())

   id    age  gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  smoke  \
0   0  18393       2     168    62.0    110     80            1     1      0   
1   1  20228       1     156    85.0    140     90            3     1      0   
2   2  18857       1     165    64.0    130     70            3     1      0   
3   3  17623       2     169    82.0    150    100            1     1      0   
4   4  17474       1     156    56.0    100     60            1     1      0   

   alco  active  CVD  
0     0       1    0  
1     0       1    1  
2     0       0    1  
3     0       1    1  
4     0       0    0  


In [8]:
df = df.replace({
    "gender": {1: "male", 2: "female"},
    "cholesterol": {1: "normal", 2: "borderline high", 3: "elevated"},
    "gluc": {1: "normal", 2: "pre diabetic", 3: "diabetic"},
    "smoke": {0:"no-smoke", 1: "smoke"},
    "alco": {0: "non-alco", 1: "alco"},
    "active": {0: "non-active", 1: "active"},
    "CVD": {0: "non-cvd", 1: "cvd"}
})


In [9]:
df["age"] = (df["age"] / 365).astype(int)

In [10]:
print(df.age)

0        50
1        55
2        51
3        48
4        47
         ..
69995    52
69996    61
69997    52
69998    61
69999    56
Name: age, Length: 70000, dtype: int64


In [11]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  object 
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  object 
 8   gluc         70000 non-null  object 
 9   smoke        70000 non-null  object 
 10  alco         70000 non-null  object 
 11  active       70000 non-null  object 
 12  CVD          70000 non-null  object 
dtypes: float64(1), int64(5), object(7)
memory usage: 6.9+ MB
None


### Part 2: Data Cleaning (Preprocessing)

####  1. Checking for Missing Values

In [12]:
print(df.isnull().sum())

id             0
age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
CVD            0
dtype: int64


#### 2. Checking for Duplicates

In [13]:
print(df.duplicated().sum())

0


#### 3. Handling Outliers (Invalid Ranges)

##### filter the dataset based on medically valid ranges

In [14]:
df = df[(df["age"] >= 18) & (df["age"] <= 100)]

df = df[(df["height"] >= 140) & (df["height"] <= 210)]

df = df[(df["weight"] >= 40) & (df["weight"] <= 200)]

df = df[(df["ap_hi"] >= 80) & (df["ap_hi"] <= 250)]

df = df[(df["ap_lo"] >= 50) & (df["ap_lo"] <= 150)]

### Part 3: Feature Engineering (Adding New Columns)

#### 1. Body Mass Index (BMI) 

In [15]:
# Calculate BMI
df["BMI"] = df["weight"] / ((df["height"] / 100) ** 2)

In [16]:
df["BMI"] = df["BMI"].round(2)

In [17]:
# BMI classification
def bmi_class(bmi):
    if bmi < 18.5:
        return "Under weight"
    elif 18.5 <= bmi <= 24.9:
        return "Normal weight"
    elif 25 <= bmi <= 29.9:
        return "Over weight"
    else:
        return "Obese"

df["BMI_Class"] = df["BMI"].apply(bmi_class)

#### 2. Blood Pressure Classification

In [18]:
# Function to classify blood pressure
def bp_class(row):
    sbp = row["ap_hi"]
    dbp = row["ap_lo"]
    
    if sbp < 120 and dbp < 80:
        return "Normal"
    elif 120 <= sbp <= 129 and dbp < 80:
        return "Elevated"
    elif (130 <= sbp <= 139) or (80 <= dbp <= 89):
        return "Hypertension Stage 1"
    else:
        return "Hypertension Stage 2"

# Apply the function
df["BP_Class"] = df.apply(bp_class, axis=1)

#### 3. Age Distribution

In [19]:
bins = [29, 34, 39, 44, 49, 54, 59, 64]
labels = ['29-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64']
df['age_distribution'] = pd.cut(df['age'], bins=bins, labels=labels, include_lowest=True)

In [22]:
print(df[['age', 'age_distribution']].head())

   age age_distribution
0   50            50-54
1   55            55-59
2   51            50-54
3   48            45-49
4   47            45-49


### Part 4: Final Verification

##### After cleaning and adding new features, verify the dataset.

In [23]:
print(df.head(10))

   id  age  gender  height  weight  ap_hi  ap_lo      cholesterol  \
0   0   50  female     168    62.0    110     80           normal   
1   1   55    male     156    85.0    140     90         elevated   
2   2   51    male     165    64.0    130     70         elevated   
3   3   48  female     169    82.0    150    100           normal   
4   4   47    male     156    56.0    100     60           normal   
5   8   60    male     151    67.0    120     80  borderline high   
6   9   60    male     157    93.0    130     80         elevated   
7  12   61  female     178    95.0    130     90         elevated   
8  13   48    male     158    71.0    110     70           normal   
9  14   54    male     164    68.0    110     60           normal   

           gluc     smoke      alco      active      CVD    BMI  \
0        normal  no-smoke  non-alco      active  non-cvd  21.97   
1        normal  no-smoke  non-alco      active      cvd  34.93   
2        normal  no-smoke  non-alco  no

In [24]:
print(df.columns)

Index(['id', 'age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo',
       'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'CVD', 'BMI',
       'BMI_Class', 'BP_Class', 'age_distribution'],
      dtype='object')


In [25]:
df = df[[
    "id","age","age_distribution","gender","cholesterol","gluc","smoke","alco","active",
    "height","weight","BMI","BMI_Class",
    "ap_hi", "ap_lo", "BP_Class",
    "CVD"
]]

In [26]:
print(df.columns)

Index(['id', 'age', 'age_distribution', 'gender', 'cholesterol', 'gluc',
       'smoke', 'alco', 'active', 'height', 'weight', 'BMI', 'BMI_Class',
       'ap_hi', 'ap_lo', 'BP_Class', 'CVD'],
      dtype='object')


In [27]:
print(df.head())

   id  age age_distribution  gender cholesterol    gluc     smoke      alco  \
0   0   50            50-54  female      normal  normal  no-smoke  non-alco   
1   1   55            55-59    male    elevated  normal  no-smoke  non-alco   
2   2   51            50-54    male    elevated  normal  no-smoke  non-alco   
3   3   48            45-49  female      normal  normal  no-smoke  non-alco   
4   4   47            45-49    male      normal  normal  no-smoke  non-alco   

       active  height  weight    BMI      BMI_Class  ap_hi  ap_lo  \
0      active     168    62.0  21.97  Normal weight    110     80   
1      active     156    85.0  34.93          Obese    140     90   
2  non-active     165    64.0  23.51  Normal weight    130     70   
3      active     169    82.0  28.71    Over weight    150    100   
4  non-active     156    56.0  23.01  Normal weight    100     60   

               BP_Class      CVD  
0  Hypertension Stage 1  non-cvd  
1  Hypertension Stage 2      cvd  
2  Hy

In [28]:
print(df.shape)

(68530, 17)


In [29]:
df.to_excel("heart_data1_cleaned1.xlsx", index=False)